In [1]:
#!pip install instaloader
#from instaloader import Instaloader, Profile
import numpy as np
import pandas as pd
from PIL import Image
import os

In [68]:
final_df = pd.DataFrame(columns = ['Username', 'Caption', 'Date Posted', 'Number of Likes'])

In [4]:
df = pd.read_csv('MostPopularAccounts.csv')

In [5]:
top50accounts = list(df['Username'][:50])

In [6]:
top50usernames = [x[1:] for x in top50accounts]
top50usernames[:10]

['instagram',
 'cristiano',
 'kyliejenner',
 'leomessi',
 'selenagomez',
 'therock',
 'arianagrande',
 'kimkardashian',
 'beyonce',
 'khloekardashian']

In [73]:
# Download images for top accounts
# loader = Instaloader()
# for profile_name in top50usernames[:30]:  
#     profile = Profile.from_username(loader.context, profile_name)
#     num_imgs = 0
#     for count, post in enumerate(profile.get_posts()):
#         if post.typename == 'GraphImage':
#             #loader.download_post(post, profile_name)
#             #final_df.loc[len(final_df.index)] = [profile_name, post.caption, post.date, post.likes]
#             num_imgs += 1
#         if num_imgs > 9:
#             break
#         if count > 100:
#             break

In [74]:
#final_df.to_csv('final_df.csv')

In [70]:
# https://stackoverflow.com/questions/3241929/python-find-dominant-most-common-color-in-an-image

def get_dominant_color(pil_img, palette_size=16, num_colors_to_return=1):
    # Resize image to speed up processing
    img = pil_img.copy()
    img.thumbnail((100, 100))

    # Reduce colors (uses k-means internally)
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=palette_size)

    # Find the color that occurs most often
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    
    dom_colors = []
    
    for i in range(num_colors_to_return):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        dom_colors.append(dominant_color)

    return dom_colors

In [71]:
# geeksforgeeks.org/program-change-rgb-color-model-hsv-color-model/

def rgb_to_hsv(rgb_list):
 
    r, g, b = rgb_list[0], rgb_list[1], rgb_list[2]
    # R, G, B values are divided by 255
    # to change the range from 0..255 to 0..1:
    r, g, b = r / 255.0, g / 255.0, b / 255.0
 
    # h, s, v = hue, saturation, value
    cmax = max(r, g, b)    # maximum of r, g, b
    cmin = min(r, g, b)    # minimum of r, g, b
    diff = cmax-cmin       # diff of cmax and cmin.
 
    # if cmax and cmax are equal then h = 0
    if cmax == cmin:
        h = 0
     
    # if cmax equal r then compute h
    elif cmax == r:
        h = (60 * ((g - b) / diff) + 360) % 360
 
    # if cmax equal g then compute h
    elif cmax == g:
        h = (60 * ((b - r) / diff) + 120) % 360
 
    # if cmax equal b then compute h
    elif cmax == b:
        h = (60 * ((r - g) / diff) + 240) % 360
 
    # if cmax equal zero
    if cmax == 0:
        s = 0
    else:
        s = (diff / cmax) * 100
 
    # compute v
    v = cmax * 100
    return h, s, v

In [10]:
def is_blue_dominant(rgb_list):
    # 180 to 250
    for rgb in rgb_list:
        hsv = rgb_to_hsv(rgb)
        if hsv[1] + hsv[2] > 75:
            if hsv[0] > 180:
                if hsv[0] < 250:
                    return True
    return False

In [11]:
def is_red_dominant(rgb_list):
    # 345 to 15
    for rgb in rgb_list:
        hsv = rgb_to_hsv(rgb)
        if hsv[1] + hsv[2] > 75:
            if hsv[0] > 345:
                return True
            if hsv[0] < 15:
                return True
    return False

In [12]:
def is_green_dominant(rgb_list):
    # 80 to 150
    for rgb in rgb_list:
        hsv = rgb_to_hsv(rgb)
        if hsv[1] + hsv[2] > 75:
            if hsv[0] > 80:
                if hsv[0] < 150:
                    return True
    return False

In [15]:
def show_colors(color_list):
    final_array = np.full((100,100,3), [0,0,0])
    for color in color_list:
        arr = np.full((100,100,3), color)
        final_array = np.concatenate((final_array, arr))
        final_colors = Image.fromarray(final_array.astype('uint8'))
    return final_colors

In [24]:
df = pd.read_csv('final_df.csv')

In [25]:
df

,Username,Caption,Date Posted,Number of Likes
0,instagram,“I always want to be honest and relatable. I w...,9/26/2022 15:51,340347
1,instagram,"“I began dancing before I was born, in my moth...",9/22/2022 16:02,579586
2,instagram,“Never doubt the power of your unique voice. Y...,8/29/2022 15:55,475470
3,instagram,“I do not believe your income or ethnicity sho...,8/15/2022 15:47,318912
4,instagram,“When I stare into their expressions and wonde...,8/9/2022 16:00,464339
...,...,...,...,...
195,badgalriri,"2022, we coming in HOT! we bout to bring you a...",1/7/2022 17:29,1219740
196,badgalriri,when you actually lived up to your New Years r...,12/31/2021 12:03,4343222
197,badgalriri,new new @savagexfenty for the new year! availa...,12/31/2021 12:01,2852465
198,badgalriri,NaN,12/25/2021 17:33,4837111


In [26]:
mylist = list(dict.fromkeys(list(df['Username'])))
mylist

['instagram',
 'cristiano',
 'kyliejenner',
 'leomessi',
 'selenagomez',
 'therock',
 'arianagrande',
 'kimkardashian',
 'beyonce',
 'justinbieber',
 'kendalljenner',
 'natgeo',
 'taylorswift',
 'virat.kohli',
 'nickiminaj',
 'mileycyrus',
 'neymarjr',
 'kevinhart4real',
 'zendaya',
 'badgalriri']

In [27]:
file_list = []
for username in mylist:
    for filename in os.listdir(username):
        f = os.path.join(username, filename)
        if os.path.isfile(f):
            if f[-3:] == 'jpg':
                file_list.append(f)

In [29]:
red_list = []
green_list = []
blue_list = []
for file in file_list:
    im = Image.open(file)
    #im.show()
    dom_colors = get_dominant_color(im, 16, 4)
    red_list.append(is_red_dominant(dom_colors))
    green_list.append(is_green_dominant(dom_colors))
    blue_list.append(is_blue_dominant(dom_colors))

In [30]:
df['Red-Dominant'] = red_list
df['Green-Dominant'] = green_list
df['Blue-Dominant'] = blue_list

In [31]:
df

,Username,Caption,Date Posted,Number of Likes,Red-Dominant,Green-Dominant,Blue-Dominant
0,instagram,“I always want to be honest and relatable. I w...,9/26/2022 15:51,340347,True,False,False
1,instagram,"“I began dancing before I was born, in my moth...",9/22/2022 16:02,579586,False,False,False
2,instagram,“Never doubt the power of your unique voice. Y...,8/29/2022 15:55,475470,False,False,False
3,instagram,“I do not believe your income or ethnicity sho...,8/15/2022 15:47,318912,False,False,False
4,instagram,“When I stare into their expressions and wonde...,8/9/2022 16:00,464339,False,False,True
...,...,...,...,...,...,...,...
195,badgalriri,"2022, we coming in HOT! we bout to bring you a...",1/7/2022 17:29,1219740,True,False,False
196,badgalriri,when you actually lived up to your New Years r...,12/31/2021 12:03,4343222,False,False,False
197,badgalriri,new new @savagexfenty for the new year! availa...,12/31/2021 12:01,2852465,True,False,False
198,badgalriri,NaN,12/25/2021 17:33,4837111,False,False,False


In [32]:
temp_df = df[df['Username'] == 'instagram']
print(temp_df['Number of Likes'].mean())
print(temp_df[temp_df['Red-Dominant']]['Number of Likes'].mean())
print(temp_df[temp_df['Green-Dominant']]['Number of Likes'].mean())
print(temp_df[temp_df['Blue-Dominant']]['Number of Likes'].mean())

408913.0
404100.5
431158.0
358251.0


In [33]:
mylist

['instagram',
 'cristiano',
 'kyliejenner',
 'leomessi',
 'selenagomez',
 'therock',
 'arianagrande',
 'kimkardashian',
 'beyonce',
 'justinbieber',
 'kendalljenner',
 'natgeo',
 'taylorswift',
 'virat.kohli',
 'nickiminaj',
 'mileycyrus',
 'neymarjr',
 'kevinhart4real',
 'zendaya',
 'badgalriri']

In [34]:
result_df = pd.DataFrame()
result_df['Username'] = mylist

In [35]:
avg_likes = []
avg_likes_red = []
avg_likes_green = []
avg_likes_blue = []

for username in mylist:
    temp_df = df[df['Username'] == username]
    avg_likes.append(round(temp_df['Number of Likes'].mean(), 0))
    avg_likes_red.append(round(temp_df[temp_df['Red-Dominant']]['Number of Likes'].mean(), 0))
    avg_likes_green.append(round(temp_df[temp_df['Green-Dominant']]['Number of Likes'].mean(), 0))
    avg_likes_blue.append(round(temp_df[temp_df['Blue-Dominant']]['Number of Likes'].mean(), 0))

In [36]:
result_df['Average Number of Likes'] = avg_likes
result_df['Average Number of Likes on Red-Dominant Posts'] = avg_likes_red
result_df['Average Number of Likes on Green-Dominant Posts'] = avg_likes_green
result_df['Average Number of Likes on Blue-Dominant Posts'] = avg_likes_blue

In [37]:
result_df

,Username,Average Number of Likes,Average Number of Likes on Red-Dominant Posts,Average Number of Likes on Green-Dominant Posts,Average Number of Likes on Blue-Dominant Posts
0,instagram,408913.0,404100.0,431158.0,358251.0
1,cristiano,6084814.0,5621742.0,4744913.0,5986831.0
2,kyliejenner,3935128.0,5362514.0,NaN,3123188.0
3,leomessi,3831312.0,4525080.0,3978799.0,3578974.0
4,selenagomez,2107895.0,2139540.0,NaN,1445466.0
5,therock,806595.0,306548.0,NaN,919318.0
6,arianagrande,4954693.0,5683963.0,NaN,5675917.0
7,kimkardashian,1184215.0,711645.0,282790.0,516606.0
8,beyonce,1785802.0,2322900.0,1262102.0,1479852.0
9,justinbieber,1722280.0,1429169.0,NaN,3303895.0


In [38]:
result_df['Red-Dominant Percentage'] = result_df['Average Number of Likes on Red-Dominant Posts'] / result_df['Average Number of Likes']
result_df['Green-Dominant Percentage'] = result_df['Average Number of Likes on Green-Dominant Posts'] / result_df['Average Number of Likes']
result_df['Blue-Dominant Percentage'] = result_df['Average Number of Likes on Blue-Dominant Posts'] / result_df['Average Number of Likes']

In [39]:
result_df

,Username,Average Number of Likes,Average Number of Likes on Red-Dominant Posts,Average Number of Likes on Green-Dominant Posts,Average Number of Likes on Blue-Dominant Posts,Red-Dominant Percentage,Green-Dominant Percentage,Blue-Dominant Percentage
0,instagram,408913.0,404100.0,431158.0,358251.0,0.988230,1.054400,0.876106
1,cristiano,6084814.0,5621742.0,4744913.0,5986831.0,0.923897,0.779796,0.983897
2,kyliejenner,3935128.0,5362514.0,NaN,3123188.0,1.362729,NaN,0.793669
3,leomessi,3831312.0,4525080.0,3978799.0,3578974.0,1.181078,1.038495,0.934138
4,selenagomez,2107895.0,2139540.0,NaN,1445466.0,1.015013,NaN,0.685739
5,therock,806595.0,306548.0,NaN,919318.0,0.380052,NaN,1.139752
6,arianagrande,4954693.0,5683963.0,NaN,5675917.0,1.147188,NaN,1.145564
7,kimkardashian,1184215.0,711645.0,282790.0,516606.0,0.600942,0.238800,0.436243
8,beyonce,1785802.0,2322900.0,1262102.0,1479852.0,1.300760,0.706742,0.828676
9,justinbieber,1722280.0,1429169.0,NaN,3303895.0,0.829812,NaN,1.918326


In [40]:
result_df['Red-Dominant Percentage'].mean()

0.9596152651210478

In [41]:
result_df['Green-Dominant Percentage'].mean()

0.9265671045011179

In [42]:
result_df['Blue-Dominant Percentage'].mean()

1.0153694066105419

In [72]:
#result_df.to_csv('result_df')